In [0]:
!pip3 install arviz
!pwd

# 状態空間モデル



In [0]:
import os 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

import pystan
import scipy.stats as ss

from criterion import Criterion
import utils

import arviz
import pickle

from IPython.display import display


plt.style.use("ggplot")

In [0]:
data = pd.read_csv("input/data-ss1.txt")

In [0]:
display(data.head())
display(data.describe())
plt.plot(data["X"], data["Y"])

In [0]:
base_dir = Path.cwd() / "model"
stan_path = base_dir / "stanmodel" / "model12-2.stan"
pkl_path = base_dir / "model_pkl" / "model12-2.pkl"

sm12_2 = utils.read_stanmodel(stan_path, pickle_path=pkl_path)

In [0]:
standata = {
    "T":data.shape[0],
    "Y":data["Y"],
    "T_pred":3
}

In [0]:
fit12_2 = sm12_2.sampling(
    data=standata,
    iter=3000,
    seed=496,
    warmup=300,
    chains=4
)

In [0]:
print(fit12_2)

In [0]:
ms = fit12_2.extract()

In [0]:
arviz.plot_posterior(ms["s_mu"]);

In [0]:
arviz.plot_posterior(ms["s_Y"]);

In [0]:
ms["y_pred"]

In [0]:
x = data["X"].to_list()
# x = []
for i in range(3):    
    x.append(i+22)
plt.figure(figsize=(10, 8))
plt.plot(data["X"], data["Y"], marker="o")
lower80 , upper80 = ss.mstats.mquantiles(ms["mu_all"], [0.1, 0.9], axis=0)
lower50 , upper50 = ss.mstats.mquantiles(ms["mu_all"], [0.25, 0.75], axis=0)
plt.plot(x, np.mean(ms["mu_all"], axis=0))
plt.fill_between(x, lower80, upper80, alpha=0.3)
plt.fill_between(x, lower50, upper50, alpha=0.4)
plt.xlabel("Time(Day)")
plt.ylabel("Y")
plt.show()

In [0]:
base_dir = Path.cwd() / "model"
stan_path = base_dir / "stanmodel" / "model12-3.stan"
pkl_path = base_dir / "model_pkl" / "model12-3.pkl"

sm12_3 = utils.read_stanmodel(stan_path, pickle_path=pkl_path)

In [0]:
fit12_3 = sm12_3.sampling(
    data=standata,
    iter=3000,
    seed=496,
    warmup=300,
    chains=4
)

In [0]:
print(fit12_3)

In [0]:
ms12_3 = fit12_3.extract()

In [0]:
plt.figure(figsize=(10, 8))
plt.plot(data["X"], data["Y"], marker="o")
lower80 , upper80 = ss.mstats.mquantiles(ms12_3["mu_all"], [0.1, 0.9], axis=0)
lower50 , upper50 = ss.mstats.mquantiles(ms12_3["mu_all"], [0.25, 0.75], axis=0)
plt.plot(x, np.mean(ms12_3["mu_all"], axis=0))
plt.fill_between(x, lower80, upper80, alpha=0.3)
plt.fill_between(x, lower50, upper50, alpha=0.4)
plt.xlabel("Time(Day)")
plt.ylabel("Y")
plt.show()

In [0]:
plt.figure(figsize=(10, 8))
def plot_inference_interval(original_x,
                            original_y,
                            inference_mean,
                            pred_x):
    """
    orginal_x : 解析に使ったデータのX
    original_y :　解析に使ったデータのY
    inference_mean : stanで求めた平均
    pred_x :　inference_meanの数
    """
    # draw
    plt.plot(original_x, original_y)
    lower80 , upper80 = ss.mstats.mquantiles(inference_mean, [0.1, 0.9], axis=0)
    lower50 , upper50 = ss.mstats.mquantiles(inference_mean, [0.25, 0.75], axis=0)
    plt.plot(pred_x, np.mean(inference_mean, axis=0))
    plt.fill_between(pred_x, lower80, upper80, alpha=0.3)
    plt.fill_between(pred_x, lower50, upper50, alpha=0.4)
    plt.xlabel("Time")
    plt.ylabel("Y")
    plt.show()

## 季節調節項




In [0]:
data = pd.read_csv(Path.cwd()/"input"/"data-ss2.txt")

In [0]:
display(data.head())
display(data.describe())

In [0]:
plt.plot(data["X"], data["Y"], marker="o")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [0]:
standata ={
    "T":np.max(data["X"]),
    "Y":data["Y"]
}

In [0]:
stan_path = base_dir / "stanmodel" / "model12-6.stan"
pkl_path = base_dir / "model_pkl" / "model12-6.pkl"

sm12_6 = utils.read_stanmodel(stan_path, pkl_path)

In [0]:
fit12_6 = sm12_6.sampling(
    data=standata,
    iter=4000,
    warmup=400,
    seed=496
)

In [0]:
print(fit12_6)

In [0]:
plt.figure(figsize=(10,8))
plot_inference_interval(original_x=data["X"],
                        original_y=data["Y"],
                        inference_mean=fit12_6.extract()["mu"],
                        pred_x=data["X"])

In [0]:
plt.figure(figsize=(10,8))
plot_inference_interval(original_x=data["X"],
                        original_y=data["Y"],
                        inference_mean=fit12_6.extract()["y_mean"],
                        pred_x=data["X"])

# 変化点検知

ノイズが従う分布を正規分布から違う分布を使って拡張する。

ここではコーシー分布が使われる。


In [0]:
base_dir = Path.cwd() 
data = pd.read_csv(base_dir / "input" / "data-changepoint.txt")

In [0]:
display(data.head())
display(data.describe())

In [0]:
plt.plot(data["X"], data["Y"])
plt.xlabel("Time(second)")
plt.ylabel("Y")
plt.show()

$$\mu[t] \sim Cauchy(\mu[t-1], \sigma_{\mu})\tag{1}$$
$$Y[t] \sim Normal(\mu[t], \sigma_{Y}\tag{2})$$

In [0]:
stan_path = base_dir / "model" / "stanmodel" / "model12-7.stan"
pkl_path = base_dir / "model" / "model_pkl" / "model12-7.pkl"

sm12_7 = utils.read_stanmodel(stan_path, pkl_path)

standata = {
    "T":np.max(data["X"]),
    "Y":data["Y"],
}

In [0]:
%%time

fit12_7 = sm12_7.sampling(
    data=standata,
    iter=4000,
    warmup=400,
    seed=496,
)

In [0]:
print(fit12_7)

In [0]:
plt.figure(figsize=(10,8))
plot_inference_interval(original_x=data["X"],
                        original_y=data["Y"],
                        inference_mean=fit12_7.extract()["mu"],
                        pred_x=data["X"])

# 1次元空間構造

In [0]:
data_path = base_dir / "input" / "data-kubo11a.txt"
data = pd.read_csv(data_path)

In [0]:
display(data.head())
display(data.describe())

In [0]:
plt.figure(figsize=(10,8))
plt.scatter(np.linspace(1, 51, 50), data["Y"])
plt.xlabel("i")
plt.ylabel("Y")
plt.show()

$$
p(r[1], r[2], \dots , r[I]) \propto \frac{1}{\sigma_{r}^{A}} exp \left[- \frac{1}{2 \sigma_{r}^{2}} \displaystyle{\sum_{i=2}^{I}}(r[i] - r[i-1])^2 \right] \tag{12.9}
$$
$$
Y[i] \sim Poisson(exp(r[i])) \ \ \ i =1, \dots , I \tag{12.10}
$$

In [0]:
stan_path = base_dir / "model" / "stanmodel" / "model12-11.stan"
pkl_path = base_dir / "model" / "model_pkl" / "model12-11.pkl"

sm12_11 = utils.read_stanmodel(stan_path, pkl_path)

standata = {
    "I":data.shape[0],
    "Y":data["Y"]
}

In [0]:
%%time
fit12_11 = sm12_11.sampling(
    data=standata,
    iter=4000,
    warmup=400,
    chains=4,
    seed=496
)

In [0]:
print(fit12_11)

In [0]:
plt.figure(figsize=(10,8))
utils.plot_inference_interval(original_x=np.linspace(1,51, 50),
                            original_y=data["Y"],
                            inference_mean=fit12_11.extract()["Y_mean"],
                            pred_x=np.linspace(1,51,50))

In [0]:
stan_path = base_dir / "model" / "stanmodel" / "model12-12.stan"
pkl_path = base_dir / "model" / "model_pkl" / "model12-12.pkl"

sm12_12 = utils.read_stanmodel(stan_path, pkl_path)

In [0]:
%%time 
fit12_12 = sm12_12.sampling(
    data=standata,
    iter=4000,
    warmup=400,
    seed=496,
)

In [0]:
print(fit12_12)

In [0]:
plt.figure(figsize=(10,8))
utils.plot_inference_interval(original_x=np.linspace(1,51, 50),
                            original_y=data["Y"],
                            inference_mean=fit12_12.extract()["Y_mean"],
                            pred_x=np.linspace(1,51,50))